# Dynamic Trade Generation (Inside Simulation Loop)

This notebook demonstrates how to generate external trades **dynamically during backtesting** based on current portfolio state.

## Why Dynamic Generation?

Dynamic trade generation allows your strategy to:
- React to portfolio performance and volatility
- Adjust positions based on current holdings
- Implement conditional trading logic (e.g., rebalance only on month-end)
- Use historical returns and PnL to inform decisions
- Model realistic trading behavior

This is more realistic than pre-generating all trades, as it mimics how traders actually make decisions.

In [9]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from backtesting import (
    Backtester, BacktestConfig, DataManager,
    create_simple_signal_generator,
    TargetWeightSignalGenerator,
    AlphaSignalGenerator,
    ConditionalSignalGenerator,
    TradeGeneratorConfig,
    generate_external_trades_from_signals
)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Common Issues & Solutions

### 1. KeyError when accessing DataFrame columns

**Error:** `KeyError: "None of [Index(['date', 'num_trades', ...`

**Cause:** The DataFrame is empty (no external trades were generated)

**Solution:** Always check if DataFrame is empty:
```python
trades_by_date = results.get_external_trades_by_date()
if not trades_by_date.empty:
    print(trades_by_date[['date', 'num_trades']])
else:
    print("No external trades found")
```

### 2. No trades generated

**Possible reasons:**
- Tickers in your signals don't exist in the data
- Conditional function never returns True (e.g., no month-ends in date range)
- Signal function returns empty dict
- Positions don't need rebalancing (already at target)

**Debug by adding prints:**
```python
def my_signals(context):
    signals = {'AAPL': 0.3, 'MSFT': 0.2}
    print(f"Date: {context['date']}, Signals: {signals}")  # Debug
    return signals
```

### 3. Performance issues

See the "Performance Optimization" section above for tips on speeding up backtests.

## Setup

**Important:** This notebook requires data files in the `../data/` directory.

If you don't have data files yet, **please run notebook `01_basic_setup_and_data_loading.ipynb` first** to generate sample data.

Alternatively, you can place your own CSV files in the `data` directory with these formats:
- `prices.csv` - Date index, ticker columns with closing prices
- `adv.csv` - Date index, ticker columns with average daily volume
- `betas.csv` - Date index, ticker columns with market betas (optional)
- `sectors.csv` - Ticker to sector mapping (optional)

In [10]:
# Initialize data manager and backtester
data_manager = DataManager('../sample_data')

# Configuration for faster backtesting
config = BacktestConfig(
    initial_cash=1000000,
    tc_fixed=0.001,  # 10 bps fixed cost
    max_portfolio_variance=None,  # Disable optimization for speed (set to 0.015 to enable)
    use_float32=True  # Use float32 for memory efficiency
)

backtester = Backtester(config, data_manager)

print("Backtester initialized")
print("\nPerformance Tips:")
print("- Use shorter date ranges for testing (e.g., 1-3 months)")
print("- Set max_portfolio_variance=None to disable optimization")
print("- Trade fewer tickers to reduce computation")
print("- The first run may be slower due to data loading")

Backtester initialized

Performance Tips:
- Use shorter date ranges for testing (e.g., 1-3 months)
- Set max_portfolio_variance=None to disable optimization
- Trade fewer tickers to reduce computation
- The first run may be slower due to data loading


## Example 1: Simple Callable Function

The simplest way - pass a function that generates trades based on current state.

## Performance Optimization

Dynamic trade generation can be slower than pre-generated trades because:

1. **Signal calculation overhead** - Your function runs on every trading day
2. **Optimization** - If `max_portfolio_variance` is set, the optimizer runs daily
3. **Data lookups** - Context building and price lookups happen each day

### Speed Optimization Tips:

**For Testing/Development:**
- ✅ Use **shorter date ranges** (1-3 months instead of 1 year)
- ✅ Set `max_portfolio_variance=None` to **disable optimization**
- ✅ Trade **fewer tickers** (3-5 instead of 10+)
- ✅ Use `show_progress=True` to see progress bar

**For Production:**
- Cache expensive computations in your signal function
- Use vectorized numpy operations
- Consider pre-computing signals and using static generation for long backtests
- Profile your signal function to find bottlenecks

### Typical Performance:
- **Short backtest** (2-3 months, 3 tickers, no optimization): ~1-5 seconds
- **Medium backtest** (6 months, 5 tickers, no optimization): ~5-15 seconds  
- **Long backtest** (1 year, 10 tickers, with optimization): ~30-60 seconds

The examples below use optimized settings for faster execution.

In [11]:
def generate_daily_trades(context):
    """
    Generate trades based on portfolio performance.
    
    Strategy: Increase exposure when portfolio is up, reduce when down.
    """
    # Need at least 10 days of history
    if len(context['daily_returns']) < 10:
        return {}
    
    # Calculate recent performance
    recent_return = sum(context['daily_returns'][-10:])
    
    # Adjust target weights based on performance
    # Using actual tickers from sample_data (STOCK0000, STOCK0001, etc.)
    if recent_return > 0.05:  # Up 5%
        target_weights = {
            'STOCK0000': 0.35,
            'STOCK0001': 0.30,
            'STOCK0002': 0.25
        }
    elif recent_return < -0.05:  # Down 5%
        target_weights = {
            'STOCK0000': 0.15,
            'STOCK0001': 0.15,
            'STOCK0002': 0.10
        }
    else:
        target_weights = {
            'STOCK0000': 0.25,
            'STOCK0001': 0.20,
            'STOCK0002': 0.15
        }
    
    # Convert to trades
    trades = generate_external_trades_from_signals(
        signals=target_weights,
        current_positions=context['portfolio'].positions,
        close_prices=context['prices'],
        portfolio_value=context['portfolio_value'],
        signal_type='weights'
    )
    
    return trades

# Run backtest with dynamic generation
# NOTE: Using shorter date range (2 months) for faster execution
print("Running backtest with simple callable...")
print("Using 2-month period for faster demo\n")

results = backtester.run(
    start_date=pd.Timestamp('2023-01-02'),
    end_date=pd.Timestamp('2023-03-01'),  # Reduced from 6 months to 2 months
    use_case=3,
    inputs={'external_trades': generate_daily_trades},
    show_progress=True  # Show progress bar
)

# Results
metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")

Running backtest with simple callable...
Using 2-month period for faster demo


Starting Backtest - Use Case 3
Period: 2023-01-02 to 2023-03-01

Loading data...
Loaded prices: 1250 dates, 1500 securities
Loaded ADV: 1250 dates, 1500 securities
Loaded factor exposures: 1250 dates, 1500 securities, 5 factors
Loaded factor returns: 1250 dates, 5 factors
Loaded factor covariance: (5, 5)
Loaded specific variance: 1250 dates, 1500 securities
Trading days: 43
Initial portfolio value: $1,000,000.00



Simulating: 100%|██████████| 43/43 [00:00<00:00, 82.26it/s]


Backtest Complete
Final portfolio value: $944,145.66


=== Performance Metrics ===
Total Return: -5.59%
Sharpe Ratio: -1.92
Max Drawdown: 9.62%


## Example 2: Using create_simple_signal_generator

Convenient wrapper for quick setup with signal history tracking.

In [12]:
def my_signals(context):
    """Calculate target weights based on volatility."""
    if len(context['daily_returns']) < 20:
        return {}
    
    # Calculate recent volatility
    recent_vol = np.std(context['daily_returns'][-20:]) * np.sqrt(252)
    
    # Adjust exposure based on volatility
    if recent_vol > 0.25:  # High vol - reduce exposure
        base_weight = 0.15
    elif recent_vol < 0.15:  # Low vol - increase exposure
        base_weight = 0.30
    else:
        base_weight = 0.20
    
    # Using actual tickers from sample_data
    return {
        'STOCK0000': base_weight,
        'STOCK0001': base_weight * 0.9,
        'STOCK0002': base_weight * 0.8,
        'STOCK0003': base_weight * 0.7
    }

# Create signal generator
signal_gen = create_simple_signal_generator(
    signal_function=my_signals,
    signal_type='weights'
)

print("Running backtest with signal generator...")
results = backtester.run(
    start_date=pd.Timestamp('2023-01-02'),
    end_date=pd.Timestamp('2023-06-30'),
    use_case=3,
    inputs={'external_trades': signal_gen}
)

# Results
metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")

# View signal history
signal_history = signal_gen.get_history()
print(f"\nGenerated {len(signal_history)} signals across {signal_history['date'].nunique()} days")
print("\nSample signals:")
print(signal_history.head(10))

Running backtest with signal generator...

Starting Backtest - Use Case 3
Period: 2023-01-02 to 2023-06-30

Loading data...
Trading days: 130
Initial portfolio value: $1,000,000.00



Simulating: 100%|██████████| 130/130 [00:01<00:00, 80.23it/s]


Backtest Complete
Final portfolio value: $1,070,108.10


=== Performance Metrics ===
Total Return: 7.01%
Sharpe Ratio: 0.76

Generated 444 signals across 111 days

Sample signals:
        date     ticker  signal  has_trade
0 2023-01-27  STOCK0000    0.30       True
1 2023-01-27  STOCK0001    0.27       True
2 2023-01-27  STOCK0002    0.24       True
3 2023-01-27  STOCK0003    0.21       True
4 2023-01-30  STOCK0000    0.30       True
5 2023-01-30  STOCK0001    0.27       True
6 2023-01-30  STOCK0002    0.24       True
7 2023-01-30  STOCK0003    0.21       True
8 2023-01-31  STOCK0000    0.30       True
9 2023-01-31  STOCK0001    0.27       True


## Example 3: Target Weight Signal Generator

More control with custom configuration.

In [13]:
def rebalancing_weights(context):
    """
    Rebalance when weights drift too much from target.
    """
    # Target allocation using actual tickers from sample_data
    target_allocation = {
        'STOCK0000': 0.25,
        'STOCK0001': 0.25,
        'STOCK0002': 0.20,
        'STOCK0003': 0.15,
        'STOCK0004': 0.15
    }
    
    # Calculate current weights
    portfolio_value = context['portfolio_value']
    current_positions = context['portfolio'].positions
    prices = context['prices']
    
    current_weights = {}
    for ticker, shares in current_positions.items():
        if ticker in prices:
            value = shares * prices[ticker]
            current_weights[ticker] = value / portfolio_value
    
    # Check if rebalancing is needed (5% drift threshold)
    needs_rebalance = False
    for ticker, target_weight in target_allocation.items():
        current_weight = current_weights.get(ticker, 0)
        if abs(current_weight - target_weight) > 0.05:
            needs_rebalance = True
            break
    
    return target_allocation if needs_rebalance else {}

# Configure trade generation with multiple fills
trade_config = TradeGeneratorConfig(
    price_impact_bps=5.0,
    num_fills_per_ticker=3,  # Simulate VWAP
    max_adv_participation=0.10
)

signal_gen = TargetWeightSignalGenerator(
    signal_function=rebalancing_weights,
    trade_generator_config=trade_config
)

print("Running backtest with target weight generator...")
results = backtester.run(
    start_date=pd.Timestamp('2023-01-02'),
    end_date=pd.Timestamp('2023-06-30'),
    use_case=3,
    inputs={'external_trades': signal_gen}
)

metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")

# Analyze trades
trades_summary = results.get_external_trades_summary()
if not trades_summary.empty:
    print("\n=== External Trades Summary ===")
    print(trades_summary)
else:
    print("\nNo external trades summary available.")


Running backtest with target weight generator...

Starting Backtest - Use Case 3
Period: 2023-01-02 to 2023-06-30

Loading data...
Trading days: 130
Initial portfolio value: $1,000,000.00



Simulating: 100%|██████████| 130/130 [00:01<00:00, 85.48it/s]


Backtest Complete
Final portfolio value: $1,095,763.53


=== Performance Metrics ===
Total Return: 9.58%
Sharpe Ratio: 0.75

=== External Trades Summary ===
      ticker  num_trades  total_qty        vwap   avg_price  total_cost
0  STOCK0000           6       3030   81.407644   74.373131  253.429512
1  STOCK0001           6       1342  189.804352  171.656261  254.853055
2  STOCK0002           6       1067  147.977230  151.325541  242.356357
3  STOCK0003           6       1173  124.410981  114.227604  154.105133
4  STOCK0004           6       4794   36.698956   32.641637  175.989578



/Users/sercan/repo/github/fbny/backtesting/notebooks/../backtesting/results.py:363: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwap = external_trades.groupby('ticker').apply(


def is_month_end(context):
    """Check if today is month end."""
    date = context['date']
    return date.is_month_end

def monthly_weights(context):
    """Target weights for monthly rebalancing."""
    return {
        'AAPL': 0.30,
        'MSFT': 0.25,
        'GOOGL': 0.20,
        'AMZN': 0.15,
        'TSLA': 0.10
    }

signal_gen = ConditionalSignalGenerator(
    signal_function=monthly_weights,
    condition_function=is_month_end,
    signal_type='weights'
)

print("Running backtest with conditional generator (month-end only)...")
results = backtester.run(
    start_date=pd.Timestamp('2023-01-01'),
    end_date=pd.Timestamp('2023-06-30'),
    use_case=3,
    inputs={'external_trades': signal_gen},
    show_progress=True
)

metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")

# Check trading days
trades_by_date = results.get_external_trades_by_date()

if not trades_by_date.empty:
    print(f"\nTraded on {len(trades_by_date)} days (should be ~6 month-ends)")
    print("\nTrading dates:")
    # Check which columns actually exist
    available_cols = trades_by_date.columns.tolist()
    cols_to_show = [c for c in ['date', 'num_trades', 'num_tickers', 'total_notional'] if c in available_cols]
    print(trades_by_date[cols_to_show])
else:
    print("\nNo external trades were generated.")
    print("This could mean:")
    print("- The condition function never returned True")
    print("- There were no month-end dates in the date range")
    print("- The tickers in the signals don't exist in your data")

In [14]:
def is_month_end(context):
    """Check if today is month end."""
    date = context['date']
    return date.is_month_end

def monthly_weights(context):
    """Target weights for monthly rebalancing."""
    # Using actual tickers from sample_data
    return {
        'STOCK0000': 0.30,
        'STOCK0001': 0.25,
        'STOCK0002': 0.20,
        'STOCK0003': 0.15,
        'STOCK0004': 0.10
    }

signal_gen = ConditionalSignalGenerator(
    signal_function=monthly_weights,
    condition_function=is_month_end,
    signal_type='weights'
)

print("Running backtest with conditional generator (month-end only)...")
results = backtester.run(
    start_date=pd.Timestamp('2023-01-02'),
    end_date=pd.Timestamp('2023-06-30'),
    use_case=3,
    inputs={'external_trades': signal_gen}
)

metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")

# Check trading days
trades_by_date = results.get_external_trades_by_date()
print(f"\nTraded on {len(trades_by_date)} days (should be ~6 month-ends)")


Running backtest with conditional generator (month-end only)...

Starting Backtest - Use Case 3
Period: 2023-01-02 to 2023-06-30

Loading data...
Trading days: 130
Initial portfolio value: $1,000,000.00



Simulating: 100%|██████████| 130/130 [00:01<00:00, 81.82it/s]



Backtest Complete
Final portfolio value: $1,106,661.98


=== Performance Metrics ===
Total Return: 10.67%
Sharpe Ratio: 0.84

Traded on 5 days (should be ~6 month-ends)


## Example 5: Complex Strategy with State Awareness

A more sophisticated strategy that adjusts based on multiple factors.

In [15]:
def adaptive_strategy(context):
    """
    Adaptive strategy that adjusts weights based on:
    - Portfolio performance
    - Volatility
    - Drawdown
    """
    # Need sufficient history
    if len(context['daily_returns']) < 30:
        return {}
    
    # Calculate metrics
    recent_returns = context['daily_returns'][-30:]
    cumulative_return = np.prod([1 + r for r in recent_returns]) - 1
    volatility = np.std(recent_returns) * np.sqrt(252)
    
    # Calculate drawdown
    portfolio_values = context['portfolio_value']
    # Simplified drawdown calculation
    
    # Base weights using actual tickers from sample_data
    base_weights = {
        'STOCK0000': 0.25,
        'STOCK0001': 0.20,
        'STOCK0002': 0.20,
        'STOCK0003': 0.15,
        'STOCK0004': 0.10
    }
    
    # Adjust leverage based on volatility
    if volatility > 0.30:
        leverage = 0.6  # Reduce in high vol
    elif volatility < 0.15:
        leverage = 1.1  # Increase in low vol
    else:
        leverage = 0.9
    
    # Adjust based on performance
    if cumulative_return < -0.15:  # Down 15%
        leverage *= 0.7  # Cut exposure
    elif cumulative_return > 0.20:  # Up 20%
        leverage *= 0.85  # Take some profits
    
    # Scale weights
    target_weights = {k: v * leverage for k, v in base_weights.items()}
    
    # Check if adjustment is significant
    current_positions = context['portfolio'].positions
    prices = context['prices']
    pv = context['portfolio_value']
    
    current_weights = {}
    for ticker, shares in current_positions.items():
        if ticker in prices:
            current_weights[ticker] = (shares * prices[ticker]) / pv
    
    # Only rebalance if drift > 3%
    needs_rebalance = any(
        abs(current_weights.get(t, 0) - w) > 0.03
        for t, w in target_weights.items()
    )
    
    return target_weights if needs_rebalance else {}

# Use with advanced configuration
trade_config = TradeGeneratorConfig(
    price_impact_bps=7.0,
    num_fills_per_ticker=5,
    max_adv_participation=0.08
)

signal_gen = TargetWeightSignalGenerator(
    signal_function=adaptive_strategy,
    trade_generator_config=trade_config
)

print("Running backtest with adaptive strategy...")
results = backtester.run(
    start_date=pd.Timestamp('2023-01-02'),
    end_date=pd.Timestamp('2023-12-31'),
    use_case=3,
    inputs={'external_trades': signal_gen}
)

# Comprehensive analysis
metrics = results.calculate_metrics()
print("\n=== Performance Metrics ===")
print(f"Total Return: {metrics['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
print(f"Calmar Ratio: {metrics.get('calmar_ratio', 0):.2f}")

# PnL breakdown
pnl_df = results.get_pnl_breakdown_dataframe()
print("\n=== PnL Breakdown Summary ===")
pnl_df = results.get_pnl_breakdown_dataframe()
print("\n=== PnL Breakdown Summary ===")
if not pnl_df.empty:
    print(f"Total External PnL: ${pnl_df['external_pnl'].sum():,.0f}")
    print(f"Total Executed PnL: ${pnl_df['executed_pnl'].sum():,.0f}")
    print(f"Total Overnight PnL: ${pnl_df['overnight_pnl'].sum():,.0f}")
else:
    print("No data in pnl_df")

# Trading activity
trades_by_date = results.get_external_trades_by_date()
print(f"\nTraded on {len(trades_by_date)} days out of ~252 trading days")
trades_by_date = results.get_external_trades_by_date()
print(f"\nTraded on {len(trades_by_date)} days out of ~252 trading days")
if not trades_by_date.empty:
    print(f"Average trades per day: {trades_by_date['num_trades'].mean():.1f}")
    print(f"Total transaction costs: ${trades_by_date['total_cost'].sum():,.0f}")
else:
    print("No data in trades_by_date")

# Signal history
signal_history = signal_gen.get_history()
print(f"\nSignal history: {len(signal_history)} signals generated")

Running backtest with adaptive strategy...

Starting Backtest - Use Case 3
Period: 2023-01-02 to 2023-12-31

Loading data...
Trading days: 260
Initial portfolio value: $1,000,000.00



Simulating: 100%|██████████| 260/260 [00:03<00:00, 83.77it/s]


Backtest Complete
Final portfolio value: $1,255,641.21


=== Performance Metrics ===
Total Return: 25.56%
Sharpe Ratio: 1.05
Max Drawdown: 12.22%
Calmar Ratio: 2.02

=== PnL Breakdown Summary ===

=== PnL Breakdown Summary ===
Total External PnL: $-777
Total Executed PnL: $0
Total Overnight PnL: $258,638

Traded on 11 days out of ~252 trading days

Traded on 11 days out of ~252 trading days
Average trades per day: 24.9
Total transaction costs: $2,220

Signal history: 55 signals generated


## Example 6: Comparing Static vs Dynamic

Compare pre-generated trades vs dynamic generation.

In [16]:
from backtesting import ExternalTradeGenerator

# Static: Pre-generate all trades
print("Strategy 1: Static pre-generated trades")
print("="*50)

prices_df = data_manager.load_prices()
dates = prices_df.index[:100]  # First 100 days

# Generate all trades upfront
generator = ExternalTradeGenerator()
target_positions_by_date = {}

# Using actual tickers from sample_data
for date in dates:
    # Simple fixed allocation
    target_positions_by_date[date] = {
        'STOCK0000': 100,
        'STOCK0001': 100,
        'STOCK0002': 50
    }

static_trades = generator.generate_multi_day_trades(
    dates=dates,
    target_positions_by_date=target_positions_by_date,
    prices_df=prices_df,
    initial_positions={}
)

results_static = backtester.run(
    start_date=dates[0],
    end_date=dates[-1],
    use_case=3,
    inputs={'external_trades': static_trades}
)

metrics_static = results_static.calculate_metrics()
print(f"Return: {metrics_static['total_return']:.2%}")
print(f"Sharpe: {metrics_static['sharpe_ratio']:.2f}")
print(f"Max DD: {metrics_static['max_drawdown']:.2%}")

# Dynamic: Generate during backtest
print("\nStrategy 2: Dynamic generation (adjusts to performance)")
print("="*50)

def dynamic_positions(context):
    """Adjust positions based on recent performance."""
    if len(context['daily_returns']) < 5:
        base_size = 100
    else:
        recent_return = sum(context['daily_returns'][-5:])
        
        if recent_return > 0.02:  # Up 2%
            base_size = 120  # Increase
        elif recent_return < -0.02:  # Down 2%
            base_size = 80  # Decrease
        else:
            base_size = 100
    
    # Using actual tickers from sample_data
    return {
        'STOCK0000': base_size,
        'STOCK0001': base_size,
        'STOCK0002': base_size // 2
    }

signal_gen_dynamic = create_simple_signal_generator(
    signal_function=dynamic_positions,
    signal_type='positions'
)

results_dynamic = backtester.run(
    start_date=dates[0],
    end_date=dates[-1],
    use_case=3,
    inputs={'external_trades': signal_gen_dynamic}
)

metrics_dynamic = results_dynamic.calculate_metrics()
print(f"Return: {metrics_dynamic['total_return']:.2%}")
print(f"Sharpe: {metrics_dynamic['sharpe_ratio']:.2f}")
print(f"Max DD: {metrics_dynamic['max_drawdown']:.2%}")

# Comparison
print("\nComparison:")
print("="*50)
print(f"Dynamic outperformance: {(metrics_dynamic['total_return'] - metrics_static['total_return']) * 100:.2f}%")
print(f"\nDynamic strategy adjusted positions based on performance,")
print(f"while static used fixed positions throughout.")

Strategy 1: Static pre-generated trades

Starting Backtest - Use Case 3
Period: 2023-01-02 to 2023-05-19

Loading data...
Trading days: 100
Initial portfolio value: $1,000,000.00



Simulating: 100%|██████████| 100/100 [00:01<00:00, 80.68it/s]



Backtest Complete
Final portfolio value: $1,003,890.25

Return: 0.39%
Sharpe: 0.98
Max DD: 0.64%

Strategy 2: Dynamic generation (adjusts to performance)

Starting Backtest - Use Case 3
Period: 2023-01-02 to 2023-05-19

Loading data...
Trading days: 100
Initial portfolio value: $1,000,000.00



Simulating: 100%|██████████| 100/100 [00:01<00:00, 86.38it/s]


Backtest Complete
Final portfolio value: $1,003,890.25

Return: 0.39%
Sharpe: 0.98
Max DD: 0.64%

Comparison:
Dynamic outperformance: 0.00%

Dynamic strategy adjusted positions based on performance,
while static used fixed positions throughout.


## Summary

This notebook demonstrated:

1. **Simple callable functions** - Pass a function directly to the backtester
2. **Signal generator classes** - Use pre-built generators for common patterns
3. **Conditional trading** - Only trade when specific conditions are met
4. **State-aware strategies** - Adjust based on portfolio performance and volatility
5. **Static vs Dynamic** - Compare pre-generated vs dynamic trade generation

### Key Advantages of Dynamic Generation

- **Realistic**: Mimics how traders actually make decisions
- **Adaptive**: React to portfolio state and market conditions
- **Flexible**: Easy to implement complex conditional logic
- **Memory efficient**: No need to pre-generate all trades
- **Trackable**: Signal history available for analysis

### Next Steps

- Implement your own signal generation logic
- Test different rebalancing conditions
- Combine with risk management (Use Case 3 optimization)
- Analyze execution quality and PnL attribution
- Generate comprehensive reports with `results.generate_full_report()`